# Exercise

In this exercise we will look at a class that analyses bank statement transactions, and make improvements to increase its maintainability.

Consider the following class:

In [ ]:
from datetime import datetime

class BasicStatementAnalyser:

    def analyse(self):

        file_path = "data/bank-data-2020.csv"
        DATE_FORMAT = '%d-%m-%Y'

        with open(file_path, 'r') as content_file:
            content = content_file.readlines()

        total_amount = 0
        for line in content[1:]:
            columns = line.rstrip().split(",")
            amount = float(columns[1])
            total_amount = total_amount + amount

        print(f"The total for all transactions is {total_amount}")

        total_january = 0
        for line in content[1:]:
            columns = line.rstrip().split(",")
            date = datetime.strptime(columns[0], DATE_FORMAT)
            if date.month == 1:
                amount = float(columns[1])
                total_january = total_january + amount

        print(f"The total for transactions in January is {total_january}")

        total_february = 0
        for line in content[1:]:
            columns = line.rstrip().split(",")
            date = datetime.strptime(columns[0], DATE_FORMAT)
            if date.month == 2:
                amount = float(columns[1])
                total_february = total_february + amount

        print(f"The total for transactions in February is {total_february}")

        total_salary = 0
        for line in content[1:]:
            columns = line.rstrip().split(",")
            if columns[2] == "Salary":
                amount = float(columns[1])
                total_salary = total_salary + amount

        print(f"The total salary is {total_salary}")


BasicStatementAnalyser().analyse()

In what ways does it violate our code maintainability guidelines?

In [ ]:
# Your answers here...
# 1. The analyse() function violates the Single Responsibility Principle.
# It is currently parsing the csv, as well as making calculations.
# 2. The total for January and total for February calculations are copy pasted.
# 3. The logic for `line.rstrip().split(",")` is copy pasted.
# If the format of the lines in the csv changes, every occurence of that logic would
# need to be modified.
# 4. It is highly coupled.
# If the name or location of the csv file changes, you would need to edit how the class 
# is defined (rather than just how it is called)


See if you can rewrite the class to remedy those violations.

Some ideas to consider:
- Split out the file handling section of the `analyse` function into its own function.
- Write a function to calculate monthly total and use it to calculate January's and February's total.
- Add the file name as an input argument, so that you can test your class on `data/bank-data-2020.csv` and `data/bank-data-2021.csv`
- Reduce the duplication of `line.rstrip().split(",")`
    - Bonus marks for storing transactions as instances of a separate `BankTransaction` class.

In [ ]:
# Your code here...
# class StatementAnalyser:
class BankTransaction:
    def __init__(self, date, amount, tag):
        self.date = date
        self.amount = amount
        self.tag = tag

class StatementAnalyser:
    def _parse_from_csv(self, file_path):

        with open(file_path, 'r') as content_file:
            content = content_file.readlines()

        result = []
        for line in content[1:]:
            columns = line.rstrip().split(",")
            amount = float(columns[1])
            tag = columns[2]
            date = datetime.strptime(columns[0], DATE_FORMAT)
            result.append(BankTransaction(date, amount, tag))

        return result
    
    def calculate_month_total(self, month):
        month_total = 0
        for transaction in self.transactions:
            if transaction.date.month == month:
                month_total = month_total + transaction.amount
        return month_total

    def analyse(self, file_path):

        self.transactions = self._parse_from_csv(file_path)

        total_amount = 0
        for transaction in self.transactions:
            total_amount = total_amount + transaction.amount

        print(f"The total for all transactions is {total_amount}")

        print(f"The total for transactions in January is {self.calculate_month_total(1)}")

        print(f"The total for transactions in February is {self.calculate_month_total(2)}")

        total_salary = 0
        for transaction in self.transactions:
            if transaction.tag == "Salary":
                total_salary = total_salary + transaction.amount

        print(f"The total salary is {total_salary}")

StatementAnalyser().analyse("data/bank-data-2020.csv")
print("---")
StatementAnalyser().analyse("data/bank-data-2021.csv")